<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_LSTM_baseline0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import Libraries, setting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
tf.__version__

'2.2.0'

In [0]:
import warnings
import tensorflow as tf
import pickle
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import pprint
from tensorflow.keras.layers import Bidirectional,LSTM,Dense,Dropout,BatchNormalization,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from numpy import array
import nltk
nltk.download('punkt')

In [0]:
# we will store the params as we go along in this object
params = {}
project_path = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"
model_path = "/content/drive/My Drive/AIML-MRC-Capstone/models/"
tensorboard_logpath  = "/content/drive/My Drive/AIML-MRC-Capstone/models/tensorboard-logs/"

# Objective - LSTM Baseline 0 

*   **Inputs: A question q = {q1, ..., qQ} of length Q and a context paragraph p = {p1, ..., pP } of length P.**
*   **Output: An answer span {as, ae} where as is the index of the first answer token in p, ae is the index of the last answer token in p, 0 <= as, ae >= m, and ae >= as.** 



## 2 Load Squad Data - Cleaned and curated (output of preprocessing step)

### 2.1 Load Data

In [13]:
squad_df = pd.read_csv(project_path+'squad_data_final.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)
squad_df.head(2)

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible,clean_context,clean_question,clean_answer,answer_len,answer_end,answer_span,answer_word_span
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,269,in the late 1990s,NaN,NaN,False,beyonc giselle knowlescarter bijnse beeyonsay ...,when did beyonce start becoming popular,in the late 1990s,17,286,"(269, 286)","(-1, -1)"
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,207,singing and dancing,NaN,NaN,False,beyonc giselle knowlescarter bijnse beeyonsay ...,what areas did beyonce compete in when she was...,singing and dancing,19,226,"(207, 226)","(21, 23)"


### 2.2 Create Train, Validation and Test data

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample,shuffle

# train = resample(train)
# train = shuffle(train,n_samples =50000)

train,test = train_test_split(squad_df,test_size = 0.2)
train,val = train_test_split(train,test_size=0.25)

print(train.shape)
print(val.shape)
print(test.shape)

(78183, 16)
(26061, 16)
(26062, 16)


In [16]:
train["answer_word_span"] = train["answer_word_span"].apply(lambda x :eval(x))
test["answer_word_span"] = test["answer_word_span"].apply(lambda x :eval(x))
val["answer_word_span"] = val["answer_word_span"].apply(lambda x :eval(x))
train.info()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<class 'pandas.core.frame.DataFrame'>
Int64Index: 78183 entries, 65753 to 63112
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   title                   78183 non-null  object 
 1   context                 78183 non-null  object 
 2   question                78183 non-null  object 
 3   id                      78183 non-null  object 
 4   answer_start            78183 non-null  int64  
 5   answer                  51966 non-null  object 
 6   plausible_answer_start  26217 non-null  float64
 7   plausible_answer        26217 non-null  object 
 8   is_impossible           78183 non-null  bool   
 9   clean_context           78183 non-null  object 
 10  clean_question          78183 non-null  object 
 11  clean_answer            78183 non-null  object 
 12  answer_len              78183 non-null  int64  
 13  answer_end              78183 non-null  int64  
 14  answer_span             78183 non-

### 2.3 Build Tokenizer

In [12]:
from tqdm import tqdm
params['tokenizer_num_words'] = 80000
tokenizer = preprocessing.text.Tokenizer(num_words=params['tokenizer_num_words'])

# NOTE: tokenizer is been made out of original dataset
for text in tqdm([squad_df['clean_context'], squad_df['clean_question']]):  
  tokenizer.fit_on_texts(text.values)

# total tokenizer words
params['vocab_size'] = len(tokenizer.word_index)

### SAVE TOKENIZERS
with open(model_path + "tokenizer.pkl","wb") as f:
    pickle.dump(tokenizer,f)



100%|██████████| 2/2 [00:08<00:00,  4.39s/it]


In [0]:
tokenizer.word_index['how']

39

### 2.4 Update parameters

In [17]:
# From the EDA and historgrams we can conclude that - 
# 99% percentile of context word length = 285
# 99% percentile or question word lengt = 20
context_length = 285
question_length = 20
params['train_shape'] = train.shape
params['val_shape'] = val.shape
params['test_shape'] = test.shape
params['context_length_99'] = context_length # initialize with a high percentile
params['question_length_99'] = question_length # initialize with a high percentile
params['embedding_size'] = 100
params['rnn_units'] = 256
params['context_pad_seq'] = 'pre'
params['question_pad_seq'] = 'pre'

pprint.pprint(params)

{'context_length_99': 285,
 'context_pad_seq': 'pre',
 'embedding_size': 100,
 'question_length_99': 20,
 'question_pad_seq': 'pre',
 'rnn_units': 256,
 'test_shape': (26062, 16),
 'tokenizer_num_words': 80000,
 'train_shape': (78183, 16),
 'val_shape': (26061, 16),
 'vocab_size': 100850}


## 3 Vectorization / Encoding

#### 3.1 Integer Sequence of Context and Question 

In [0]:
train_clean_context_sequence = tokenizer.texts_to_sequences(train["clean_context"].values)
test_clean_context_sequence = tokenizer.texts_to_sequences(test["clean_context"].values)
val_clean_context_sequence = tokenizer.texts_to_sequences(val["clean_context"].values)


train_clean_question_sequence = tokenizer.texts_to_sequences(train["clean_question"].values)
test_clean_question_sequence = tokenizer.texts_to_sequences(test["clean_question"].values)
val_clean_question_sequence = tokenizer.texts_to_sequences(val["clean_question"].values)


In [0]:
train_clean_question_sequence[5:10]

[[71, 11, 33390, 6, 2134],
 [2, 1743, 1445, 263, 6388, 3343, 288],
 [2, 29, 16, 2728, 766, 147, 336, 59, 2436],
 [2, 14, 933, 41, 71, 1, 2870, 98, 1202, 98, 1875, 5123],
 [39, 9, 2617, 140, 26, 989, 563, 288]]

In [0]:
train['clean_question'][5:10]

72672                         when was mdrtb first observed
129071    what electronic charge do cellular molecules have
106489    what year did cd players become available for ...
76709     what is it called when the variations are text...
90219       how many courses does a mandolin commonly have 
Name: clean_question, dtype: object

#### 3.2 Find Max Sequence length of Context and Question

In [19]:
# max length of context
params['context_max_length'] = max(max(len(txt) for txt in train_clean_context_sequence),
                                  max(len(txt) for txt in test_clean_context_sequence),
                                  max(len(txt) for txt in val_clean_context_sequence))

params['question_max_length'] = max(max(len(txt) for txt in train_clean_question_sequence),
                                  max(len(txt) for txt in test_clean_question_sequence),
                                  max(len(txt) for txt in val_clean_question_sequence))


pprint.pprint(params)

{'context_length_99': 285,
 'context_max_length': 426,
 'context_pad_seq': 'pre',
 'embedding_size': 100,
 'question_length_99': 20,
 'question_max_length': 40,
 'question_pad_seq': 'pre',
 'rnn_units': 256,
 'test_shape': (26062, 16),
 'tokenizer_num_words': 80000,
 'train_shape': (78183, 16),
 'val_shape': (26061, 16),
 'vocab_size': 100850}


#### 3.3 Padding of the sequences

In [20]:
train_context_sequence = preprocessing.sequence.pad_sequences(train_clean_context_sequence,maxlen=params['context_max_length'])
test_context_sequence = preprocessing.sequence.pad_sequences(test_clean_context_sequence,maxlen=params['context_max_length'])
val_context_sequence = preprocessing.sequence.pad_sequences(val_clean_context_sequence,maxlen=params['context_max_length'])

print(train_context_sequence.shape)
print(test_context_sequence.shape)
print(val_context_sequence.shape)

(78183, 426)
(26062, 426)
(26061, 426)


In [21]:
train_question_sequence = preprocessing.sequence.pad_sequences(train_clean_question_sequence,maxlen=params['question_max_length'])
test_question_sequence = preprocessing.sequence.pad_sequences(test_clean_question_sequence,maxlen=params['question_max_length'])
val_question_sequence = preprocessing.sequence.pad_sequences(val_clean_question_sequence,maxlen=params['question_max_length'])

print(train_question_sequence.shape)
print(test_question_sequence.shape)
print(val_question_sequence.shape)


(78183, 40)
(26062, 40)
(26061, 40)


#### 3.4 Create Answer Sequence 

Encode y_trues as big array consisting of ans_start + ans_end. This has to be used in loss function as well. We will use the answer_word_span feature

**y_true = answer_start + answer_end**

In [0]:
# for train data
y_train = []
span_ofr = 0;
params['train_span_outofrange'] = 0
params['test_span_outofrange'] = 0
params['val_span_outofrange'] = 0

for i in range(len(train)):    
    s = np.zeros(params['context_max_length'],dtype = "int")
    e = np.zeros(params['context_max_length'],dtype = "int")
    start, end = train["answer_word_span"].iloc[i]    
    s[start] = 1
    e[end] = 1
    y_train.append(np.concatenate((s,e)))    

params['train_span_outofrange'] = span_ofr
span_ofr = 0;

# for test data
y_test = []
for i in range(len(test)):    
    s = np.zeros(params['context_max_length'],dtype = "int")
    e = np.zeros(params['context_max_length'],dtype = "int")        
    start,end = test["answer_word_span"].iloc[i]    
    s[start] = 1
    e[end] = 1
    y_test.append(np.concatenate((s,e)))

params['test_span_outofrange'] = span_ofr
span_ofr = 0;
                
# for val data
y_val = []
for i in range(len(val)):
    s = np.zeros(params['context_max_length'],dtype = "int")
    e = np.zeros(params['context_max_length'],dtype = "int")        
    start,end = val["answer_word_span"].iloc[i]    
    s[start] = 1
    e[end] = 1      
    y_val.append(np.concatenate((s,e)))

params['val_span_outofrange'] = span_ofr    

In [23]:
print(len(y_train),len(y_train[0]))
print(len(y_test),len(y_test[0]))
print(len(y_val),len(y_val[0]))

78183 852
26062 852
26061 852


In [0]:
def tokenize(sentence):
    """
    Returns tokenised words.
    """
    return nltk.word_tokenize(sentence)

def answer_span(context,ans):
    """
    This funtion returns anwer span start index and end index.
    """
    ans_token = tokenize(ans)
    print(ans_token)
    con_token = tokenize(context)
    print(con_token)
    print('21 ',con_token[21])
    print('22 ',con_token[22])
    print('23 ',con_token[23])
    print('24 ',con_token[24])
    ans_len = len(ans_token)
    
    if ans_len!=0 and ans_token[0] in con_token:
    
        indices = [i for i, x in enumerate(con_token) if x == ans_token[0]]
        print(indices)
        try:

            if(len(indices)>1):
                start = [i for i in indices if (con_token[i:i+ans_len] == ans_token) ]
                end = start[0] + ans_len - 1
                return start[0],end

            else:
                start = con_token.index(ans_token[0])
                end = start + ans_len - 1
                return start,end
        except:
            return -1,-1
    else:
        return -1,-1

def span_to_answer():
  return 0;        

### 3.5 Check 1 value

In [51]:
index = 2
answer_span(train['clean_context'].iloc[index],train['clean_answer'].iloc[index])

['eastern', 'mediterranean', 'sea']
['cyprus', 'isaprs', 'greek', 'ipa', '[', 'cipros', ']', 'turkish', 'kbrs', 'ipa', '[', 'kbs', ']', 'officially', 'republic', 'cyprus', 'greek', 'turkish', 'kbrs', 'cumhuriyeti', 'island', 'country', 'eastern', 'mediterranean', 'sea', 'coasts', 'syria', 'turkey', '[', 'e', ']', 'cyprus', 'third', 'largest', 'third', 'populous', 'island', 'mediterranean', 'member', 'state', 'european', 'union', 'located', 'south', 'turkey', 'west', 'syria', 'lebanon', 'northwest', 'israel', 'palestine', 'north', 'egypt', 'east', 'greece']
21  country
22  eastern
23  mediterranean
24  sea
[22]


(22, 24)

In [52]:
print("Ori Cont = ")
pprint.pprint(train['context'].iloc[index])
print("CLean Cont = ")
pprint.pprint(train['clean_context'].iloc[index])
print('Question = ',train['question'].iloc[index])
print('Clean Question = ',train['clean_question'].iloc[index])
print('Answer = ',train['answer'].iloc[index])
print('Clean Answer = ',train['clean_answer'].iloc[index])
print('AS,AE = ',train['answer_word_span'].iloc[index])
print("encoded ", y_train[index])

Ori Cont = 
('Cyprus (i/ˈsaɪprəs/; Greek: Κύπρος IPA: [ˈcipros]; Turkish: Kıbrıs IPA: '
 '[ˈkɯbɾɯs]), officially the Republic of Cyprus (Greek: Κυπριακή Δημοκρατία; '
 'Turkish: Kıbrıs Cumhuriyeti), is an island country in the Eastern '
 'Mediterranean Sea, off the coasts of Syria and Turkey.[e] Cyprus is the '
 'third largest and third most populous island in the Mediterranean, and a '
 'member state of the European Union. It is located south of Turkey, west of '
 'Syria and Lebanon, northwest of Israel and Palestine, north of Egypt and '
 'east of Greece.')
CLean Cont = 
('cyprus isaprs greek ipa [ cipros ] turkish kbrs ipa [ kbs ] officially '
 'republic cyprus greek turkish kbrs cumhuriyeti island country eastern '
 'mediterranean sea coasts syria turkey [ e ] cyprus third largest third '
 'populous island mediterranean member state european union located south '
 'turkey west syria lebanon northwest israel palestine north egypt east greece')
Question =  Where is Cyprus located?
Cl

In [0]:
pprint.pprint(params)

{'context_length_99': 285,
 'context_max_length': 426,
 'context_pad_seq': 'pre',
 'embedding_size': 100,
 'question_length_99': 20,
 'question_max_length': 40,
 'question_pad_seq': 'pre',
 'rnn_units': 256,
 'test_shape': (26062, 16),
 'test_span_outofrange': 0,
 'tokenizer_num_words': 80000,
 'train_shape': (78183, 16),
 'train_span_outofrange': 0,
 'training.batch_size': 64,
 'training.epochs': 25,
 'val_shape': (26061, 16),
 'val_span_outofrange': 0,
 'vocab_size': 100850}


In [0]:
print(squad_df['clean_context'][10])
print(train_context_sequence[110])
print(squad_df['clean_question'][10])
print(train_question_sequence[10])

### 3.5 Update and persist params

In [0]:
### SAVE PARAMS
# Writing to sample.json 
with open(model_path + "params.json", "w") as p: 
    p.write(json.dumps(params))

## 4 Model

**Implements a baseline 0 in Deep Learning based approach per our project synopsis. This baseline model uses the following layers **
0.   Input layer
1.   Embedding Layer
2.   List LSTM
3.   a custom Bilinear Similarity layer 
4.   Prediction Layer
5.   Output layer 



### 4.2 Building Model


**For Questions**

In [53]:
# question embedding
q_input = layers.Input(shape=(params['question_max_length'],),name="QUESTION_INPUT")
q_emb = layers.Embedding(input_dim=params['vocab_size']+1,
                  output_dim=params['embedding_size'],
                  name="QUESTION_EMBEDDING")(q_input)

# encoder 
q_output = layers.LSTM(units=params['rnn_units'], 
                     name='QUESTION_LSTM')(q_emb)
print(q_output.shape)

(None, 256)


**For Context**

In [54]:
c_input = layers.Input(shape=(params['context_max_length'],),name="CONTEXT_INPUT")

# context embedding
c_emb = layers.Embedding(input_dim=params['vocab_size']+1,
                  output_dim=params['embedding_size'],
                  name="CONTEXT_EMBEDDING")(c_input)



# exact_match
# ex_ = Input(shape=(CON_LEN,2))

# # pos tags
# pos_ = Input(shape=(CON_LEN,len(tag_to_num)+1))

# # term frequency
# term_ = Input(shape=(CON_LEN,1)) 

# concatenate input
# concat = concatenate([c_emb,ex_,pos_,term_])

c_output = layers.LSTM(params['rnn_units'],
                 name='CONTEXT_LSTM',return_sequences=True)(c_emb)

print("final output to bilinear ",c_output.shape)

final output to bilinear  (None, 426, 256)


**Bilinear Term**

In [56]:
# Reference -- https://github.com/kellywzhang/reading-comprehension/blob/master/attention.py
# bilinear term ####
print("Question context shape ",q_output.shape)
print("final o/p of context ",c_output.shape)

################ start prediction ######################
start = layers.Dense(params['rnn_units'])(q_output)

# ading time_slice to question (batch_size,1,hidden)
# shape (64,128) --> (64,1,128)
hidden_start_time_axis = tf.expand_dims(start, -1)

# squeeze remooves time slice we added before
# final shape = (batch_size,decoder_timesteps)
start_ = tf.squeeze(tf.matmul(c_output,hidden_start_time_axis),2)
    
start_ = tf.nn.softmax(start_,axis = 1)
    
################ end prediction ######################
end = layers.Dense(params['rnn_units'])(q_output)

hidden_end_time_axis = tf.expand_dims(end, -1)

# squeeze remooves time slice we added before
# final shape = (batch_size,decoder_timesteps)
end_ = tf.squeeze(tf.matmul(c_output,hidden_end_time_axis),2)
end_ = tf.nn.softmax(end_,axis=1)

prob_token_span = tf.concat((start_,end_),axis = 1)
print("Logits shape ",prob_token_span.shape)


# logits = BilinearSimilarity(UNITS)(q_cont,c_)
# Y_prob = Prediction()(logits)
# print("Logits shape ",logits.shape)

Question context shape  (None, 256)
final o/p of context  (None, 426, 256)
Logits shape  (None, 852)


**Predictions**

In [57]:
####### Prediction ### 
token_span = 20
start_prob = prob_token_span[:,:params['context_max_length']]
end_prob = prob_token_span[:,params['context_max_length']:]

# do the outer product
outer = tf.matmul(tf.expand_dims(start_prob, axis=2),tf.expand_dims(end_prob, axis=1))

outer = tf.linalg.band_part(outer, 0, token_span)

#print(outer.shape)

# start_position will have shape of (batch_size,)
start_position = tf.reduce_max(outer, axis=2)
#end position will have shape of (batch_size,)
end_position = tf.reduce_max(outer, axis=1)

y_probab = tf.concat([start_position,end_position],axis=1)

print(y_probab.shape)

(None, 852)


### 4.3 Custom Loss function

In [0]:
def logits_loss(y_true,logits):
    """
    Custom loss function which minimises log_loss.
    Referance https://stackoverflow.com/questions/50063613/add-loss-function-in-keras
    """
    
    #y_true = tf.cast(y_true,dtype=tf.int32)
    #logits = tf.cast(logits,dtype=tf.float32)
    
    # breaking the tensor into two half's to get start and end label.
    start_label = y_true[:,:params['context_max_length']]
    end_label = y_true[:,params['context_max_length']:]
    
    # braking the logits tensor into start and end part for loss calcultion.
    start_logit = logits[:,:params['context_max_length']]
    end_logit = logits[:,params['context_max_length']:]
    
    start_loss = tf.keras.backend.categorical_crossentropy(start_label,start_logit)
    end_loss = tf.keras.backend.categorical_crossentropy(end_label,end_logit)
    
#     start_loss = tf.losses.sparse_softmax_cross_entropy(labels=start_label, logits=start_logit)
#     end_loss = tf.losses.sparse_softmax_cross_entropy(labels=end_label, logits=end_logit)
    
    # as per paer
    
    loss = start_loss + end_loss
    
    return loss

### 4.4 Model Summary

In [59]:
model = Model(inputs = [q_input,c_input],outputs =y_probab)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
QUESTION_INPUT (InputLayer)     [(None, 40)]         0                                            
__________________________________________________________________________________________________
QUESTION_EMBEDDING (Embedding)  (None, 40, 100)      10085100    QUESTION_INPUT[0][0]             
__________________________________________________________________________________________________
CONTEXT_INPUT (InputLayer)      [(None, 426)]        0                                            
__________________________________________________________________________________________________
QUESTION_LSTM (LSTM)            (None, 256)          365568      QUESTION_EMBEDDING[0][0]         
______________________________________________________________________________________________

### 4.5 Model Compile

**Tensorboard Logs and Model compilation** 

In [0]:
# using tensorboard instance for callbacks
from time import time
from datetime import datetime
from tensorflow.python.keras.callbacks import TensorBoard

log_dir = tensorboard_logpath +"lstm-baseline0"
print('Tensprflow logs ',log_dir)
tensorboard = TensorBoard(log_dir=log_dir,histogram_freq=1)

# model compilation
model.compile(optimizer="adamax",loss=logits_loss,metrics=['accuracy'])

Tensprflow logs  /content/drive/My Drive/AIML-MRC-Capstone/models/tensorboard-logs/lstm-baseline0


### 4.6 Generator Function for use in Model.fit

In [0]:
## Reference 
def generator_function(length,batch_size = 64,data_type = 'Train'):
    """
    This function is generates batches of data to avoid strain on memory.
    """
    X1, X2, y = list(), list(), list()
    flag = True
    if data_type == 'Val':
        flag = False
    n = 0
    # loop forever over datapoints.
    while 1:
        for i in range(length):
            n += 1
            if flag:
                X1.append(train_question_sequence[i])
                X2.append(train_context_sequence[i])                
                y.append(y_train[i])
            else:
                X1.append(val_question_sequence[i])
                X2.append(val_context_sequence[i])                
                y.append(y_val[i])
            if n == batch_size:
                yield ((array(X1),array(X2)),array(y))
                X1,X2, y = list(), list(), list()
                n=0

### 4.7 Model Training

In [0]:
params['training.epochs']=25
params['training.batch_size']=64
params['training.train_length']=len(y_train)
params['training.val_length']=len(y_val)
params['training.train_steps']=params['training.train_length']//params['training.batch_size']
params['training.val_steps']=params['training.val_length']//32

pprint.pprint(params)

### SAVE PARAMS
# Writing to sample.json 
with open(model_path + "params.json", "w") as p: 
    p.write(json.dumps(params))

{'context_length_99': 285,
 'context_max_length': 426,
 'context_pad_seq': 'pre',
 'embedding_size': 100,
 'question_length_99': 20,
 'question_max_length': 40,
 'question_pad_seq': 'pre',
 'rnn_units': 256,
 'test_shape': (26062, 16),
 'test_span_outofrange': 0,
 'tokenizer_num_words': 80000,
 'train_shape': (78183, 16),
 'train_span_outofrange': 0,
 'training.batch_size': 64,
 'training.epochs': 25,
 'training.train_length': 78183,
 'training.train_steps': 1221,
 'training.val_length': 26061,
 'training.val_steps': 814,
 'val_shape': (26061, 16),
 'val_span_outofrange': 0,
 'vocab_size': 100850}


In [0]:
for i in range(params['training.epochs']):
    print("Epoch {} start at time ".format(i),datetime.now())
    
    train_generator = generator_function(params['training.train_length'],
                                         params['training.batch_size'])
    
    val_generator = generator_function(params['training.val_length'],
                                       32,
                                       "Val")
    model.fit(x=train_generator, epochs=1, 
                        steps_per_epoch=params['training.train_steps'],
                        verbose=1,
                        callbacks=[tensorboard],
                        validation_data=val_generator,
                        validation_steps=params['training.val_steps'])

Epoch 0 start at time  2020-05-31 02:06:29.202658
1221/1221 [==============================] - 136s 111ms/step - loss: 8.3468 - accuracy: 0.2365 - val_loss: 8.1262 - val_accuracy: 0.2791
Epoch 1 start at time  2020-05-31 02:08:48.412338
1113/1221 [==========================>...] - ETA: 10s - loss: 7.3605 - accuracy: 0.3453

### 4.6 Serialize and Persist Models

In [0]:
model.save_weights(model_path + "model_epoch_{}.h5".format(i))

### 4.7 Load existing models

In [0]:
model.load_weights(model_path + "model_epoch_24.h5")

### 4.8 Evaluation

**Eval on Test**

In [61]:
y_prediction = model.predict([test_question_sequence,test_context_sequence])

KeyboardInterrupt: ignored

In [0]:
y_prediction.shape[0]

26062

In [0]:
y_test_fixed = np.array(y_test)

In [0]:
y_test_fixed[0,:params['context_max_length']]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [0]:
start_pred = []
end_pred = []
for i in range(26062):
    start_pred.append(np.argmax(y_prediction[i,:params['context_max_length']]))
    end_pred.append(np.argmax(y_prediction[i,params['context_max_length']:]))
    
start = []
end = []
for i in range(26062):
    start.append(np.argmax(y_test_fixed[i,:params['context_max_length']]))
    end.append(np.argmax(y_test_fixed[i,params['context_max_length']:]))

In [0]:
print(start_pred[100:120])
print(end_pred[100:120])

[425, 10, 425, 425, 425, 425, 423, 425, 49, 425, 425, 425, 425, 425, 425, 0, 29, 425, 9, 425]
[425, 10, 425, 425, 425, 425, 423, 425, 49, 425, 425, 425, 425, 425, 425, 0, 29, 425, 9, 425]


In [0]:
print(start[100:120])
print(end[100:120])

[425, 425, 425, 20, 36, 3, 84, 425, 425, 425, 425, 70, 425, 19, 24, 8, 25, 74, 1, 49]
[425, 425, 425, 22, 40, 3, 85, 425, 425, 425, 425, 70, 425, 20, 26, 9, 28, 75, 1, 54]


In [0]:
y_predicted_new = np.zeros((26062,params['context_max_length']))
for i in range(26062):
    y_predicted_new[i,start_pred[i]:end_pred[i]+1] = 1
    
y_test_new = np.zeros((26062,params['context_max_length']))
for i in range(26062):
    y_test_new[i,start[i]:end[i]+1] = 1

In [0]:
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score,accuracy_score,precision_score
print("Micro f1-score on test data is ",f1_score(y_test_new,y_predicted_new,average="micro"))
print("Macro f1-score on test data is ",f1_score(y_test_new,y_predicted_new,average="macro"))
print("Accuracy on test data is ",accuracy_score(y_test_new,y_predicted_new))

Micro f1-score on test data is  0.24381819178683042
Macro f1-score on test data is  0.0029310912484228793
Accuracy on test data is  0.3332054331977592
